<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=8633d299000a39a91196ffd4f2ff0aa87609a58061b1624670cb0332ac5c96cc
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3711 sha256=29b2135e5ede7c25005ffc11bd20f6ea0bf74986b86799c7281068ee8d0e824d
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, min_price=100, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
経過時間: 386.16181540489197秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1459400,671.500000
1,Ｒｏｂｏｔ Ｈｏｍｅ,1435,建設業,755100,196.000000
2,ＩＮＰＥＸ,1605,鉱業,7933700,1840.000000
3,安藤・間,1719,建設業,630900,1123.000000
4,コムシスホールディングス,1721,建設業,552500,2815.000000
...,...,...,...,...,...
434,ＳＣＳＫ,9719,情報・通信業,1159300,2194.000000
435,富士ソフト,9749,情報・通信業,652400,4670.000000
436,ヤマダホールディングス,9831,小売業,4188100,427.600006
437,吉野家ホールディングス,9861,小売業,577300,2804.500000


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1459400,671.5
1,ＩＮＰＥＸ,1605,鉱業,7933700,1840.0
2,安藤・間,1719,建設業,630900,1123.0
3,コムシスホールディングス,1721,建設業,552500,2815.0
4,大林組,1802,建設業,3087500,1299.0
...,...,...,...,...,...
302,エヌ・ティ・ティ・データ,9613,情報・通信業,4520900,1943.0
303,カナモト,9678,サービス業,680700,2467.0
304,ＳＣＳＫ,9719,情報・通信業,1159300,2194.0
305,富士ソフト,9749,情報・通信業,652400,4670.0


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1459400,671.5
1,ＩＮＰＥＸ,1605,鉱業,7933700,1840.0
2,安藤・間,1719,建設業,630900,1123.0
3,コムシスホールディングス,1721,建設業,552500,2815.0
4,大林組,1802,建設業,3087500,1299.0
...,...,...,...,...,...
209,東京瓦斯,9531,電気・ガス業,4339200,3148.0
210,エイチ・アイ・エス,9603,サービス業,1482400,2037.0
211,カナモト,9678,サービス業,680700,2467.0
212,富士ソフト,9749,情報・通信業,652400,4670.0


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,Ｒｏｂｏｔ Ｈｏｍｅ,1435,建設業,755100,196.000000
1,オリエンタル白石,1786,建設業,551200,310.000000
2,三井住友建設,1821,建設業,814300,373.000000
3,エス・エム・エス,2175,サービス業,511300,2780.500000
4,パーソルホールディングス,2181,サービス業,714800,2759.500000
...,...,...,...,...,...
128,大阪瓦斯,9532,電気・ガス業,1843400,2200.000000
129,東邦瓦斯,9533,電気・ガス業,814200,2630.500000
130,静岡ガス,9543,電気・ガス業,736700,1095.000000
131,ヤマダホールディングス,9831,小売業,4188100,427.600006


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,エス・エム・エス,2175,サービス業,511300,2780.5
1,システナ,2317,情報・通信業,5471900,268.0
2,日本駐車場開発,2353,不動産業,4193100,212.0
3,新日本科学,2395,サービス業,503600,2022.0
4,ディー・エヌ・エー,2432,サービス業,862100,1715.0
5,ファンコミュニケーションズ,2461,サービス業,1006200,408.0
6,北の達人コーポレーション,2930,化学,2095400,240.0
7,ユーグレナ,2931,食料品,1372000,878.0
8,ビックカメラ,3048,小売業,815400,1070.0
9,ペッパーフードサービス,3053,小売業,843300,119.0


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 100円以上、5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_standard

Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,318000,2820.0
1,住石ホールディングス,1514,鉱業,1472000,347.0
2,ナカノフドー建設,1827,建設業,206300,396.0
3,土屋ホールディングス,1840,建設業,308900,245.0
4,大盛工業,1844,建設業,1006100,198.0
...,...,...,...,...,...
115,日本テレホン,9425,情報・通信業,2570700,405.0
116,ＳＥホールディングス・アンド・インキュベーションズ,9478,情報・通信業,285500,250.0
117,インプレスホールディングス,9479,情報・通信業,265900,207.0
118,愛眼,9854,小売業,227000,176.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ナカノフドー建設,1827,建設業,206300,396.0
1,土屋ホールディングス,1840,建設業,308900,245.0
2,大盛工業,1844,建設業,1006100,198.0
3,巴コーポレーション,1921,建設業,298400,524.0
4,塩水港精糖,2112,食料品,259000,213.0
...,...,...,...,...,...
83,明治海運,9115,海運業,1053700,660.0
84,日本テレホン,9425,情報・通信業,2570700,405.0
85,ＳＥホールディングス・アンド・インキュベーションズ,9478,情報・通信業,285500,250.0
86,インプレスホールディングス,9479,情報・通信業,265900,207.0


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ナカノフドー建設,1827,建設業,206300,396.0
1,土屋ホールディングス,1840,建設業,308900,245.0
2,大盛工業,1844,建設業,1006100,198.0
3,巴コーポレーション,1921,建設業,298400,524.0
4,塩水港精糖,2112,食料品,259000,213.0
5,ｆｏｎｆｕｎ,2323,情報・通信業,509200,566.0
6,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,239600,589.0
7,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,1085700,716.0
8,出前館,2484,情報・通信業,868400,460.0
9,アップルインターナショナル,2788,卸売業,406100,365.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,318000,2820.0
1,住石ホールディングス,1514,鉱業,1472000,347.0
2,ヤマト,1967,建設業,215700,913.0
3,クルーズ,2138,情報・通信業,257900,837.0
4,ｎｍｓ ホールディングス,2162,サービス業,234800,422.0
5,クックパッド,2193,サービス業,329500,159.0
6,クシム,2345,情報・通信業,541300,324.0
7,オートウェーブ,2666,小売業,1381800,189.0
8,セリア,2782,小売業,343700,2420.0
9,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,225300,1114.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,1472000,347.0
1,クルーズ,2138,情報・通信業,257900,837.0
2,クックパッド,2193,サービス業,329500,159.0
3,クシム,2345,情報・通信業,541300,324.0
4,プロパスト,3236,不動産業,283700,147.0
5,ｅｎｉｓｈ,3667,情報・通信業,942300,306.0
6,リミックスポイント,3825,情報・通信業,492400,214.0
7,ミライアル,4238,化学,227900,1551.0
8,サイバネットシステム,4312,情報・通信業,333700,805.0
9,わかもと製薬,4512,医薬品,231000,228.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,221100,906.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,252900,729.0
2,ジーエヌアイグループ,2160,医薬品,794900,1219.0
3,ソフトフロントホールディングス,2321,情報・通信業,214700,103.0
4,ウェッジホールディングス,2388,その他金融業,2597000,159.0
...,...,...,...,...,...
92,リニューアブル・ジャパン,9522,電気・ガス業,4563800,981.0
93,Ｍ＆Ａ総研ホールディングス,9552,サービス業,1064300,3555.0
94,マイクロアド,9553,サービス業,293100,4935.0
95,エアークローゼット,9557,サービス業,5854300,479.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,252900,729.0
1,ジーエヌアイグループ,2160,医薬品,794900,1219.0
2,ソフトフロントホールディングス,2321,情報・通信業,214700,103.0
3,ウェッジホールディングス,2388,その他金融業,2597000,159.0
4,ジェイテック,2479,サービス業,204600,333.0
5,ベースフード,2936,食料品,1608900,512.0
6,海帆,3133,小売業,4716800,540.0
7,アンビション ＤＸ ホールディングス,3300,不動産業,203700,940.0
8,ＧＡ ｔｅｃｈｎｏｌｏｇｉｅｓ,3491,不動産業,346300,1233.0
9,ＬｅＴｅｃｈ,3497,不動産業,215500,1216.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ウェッジホールディングス,2388,その他金融業,2597000,159.0
1,ジェイテック,2479,サービス業,204600,333.0
2,ベースフード,2936,食料品,1608900,512.0
3,アンビション ＤＸ ホールディングス,3300,不動産業,203700,940.0
4,ＬｅＴｅｃｈ,3497,不動産業,215500,1216.0
5,リネットジャパングループ,3556,小売業,424000,679.0
6,アプリックス,3727,情報・通信業,6257100,188.0
7,アララ,4015,情報・通信業,953200,609.0
8,ＷＡＣＵＬ,4173,情報・通信業,269000,808.0
9,免疫生物研究所,4570,医薬品,1526700,707.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,221100,906.0
1,Ａｉｍｉｎｇ,3911,情報・通信業,2533800,416.0
2,グローバルウェイ,3936,情報・通信業,421800,211.0
3,プラスアルファ・コンサルティング,4071,情報・通信業,997000,2745.0
4,ココナラ,4176,情報・通信業,249900,356.0
5,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,314300,1032.0
6,ＢＡＳＥ,4477,情報・通信業,1552100,243.0
7,フリー,4478,情報・通信業,683900,3110.0
8,アンジェス,4563,医薬品,631600,100.0
9,ナノキャリア,4571,医薬品,242300,191.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,Ａｉｍｉｎｇ,3911,情報・通信業,2533800,416.0
1,プラスアルファ・コンサルティング,4071,情報・通信業,997000,2745.0
2,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,314300,1032.0
3,ＢＡＳＥ,4477,情報・通信業,1552100,243.0
4,フリー,4478,情報・通信業,683900,3110.0
5,アンジェス,4563,医薬品,631600,100.0
6,ナノキャリア,4571,医薬品,242300,191.0
7,キャンバス,4575,医薬品,547700,999.0
8,カイオム・バイオサイエンス,4583,医薬品,230600,168.0
9,キッズウェル・バイオ,4584,医薬品,427600,196.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_standard_2

Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,1472000,347.0
1,大盛工業,1844,建設業,1006100,198.0
2,ｆｏｎｆｕｎ,2323,情報・通信業,509200,566.0
3,クシム,2345,情報・通信業,541300,324.0
4,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,1085700,716.0
5,出前館,2484,情報・通信業,868400,460.0
6,オートウェーブ,2666,小売業,1381800,189.0
7,石井食品,2894,食料品,1254300,286.0
8,丸善ＣＨＩホールディングス,3159,小売業,873800,348.0
9,クラウディアホールディングス,3607,繊維製品,562700,833.0


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
6,オートウェーブ,2666,小売業,1381800,189.0
10,セルシス,3663,情報・通信業,528300,673.0
12,ディー・エル・イー,3686,情報・通信業,513700,254.0
15,パス,3840,情報・通信業,537500,106.0
16,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,580500,2900.0
19,千代田化工建設,6366,建設業,1293500,346.0
20,ユニバーサルエンターテインメント,6425,機械,500600,2524.0
22,ティアック,6803,電気機器,566200,116.0
24,フェローテックホールディングス,6890,電気機器,1264400,3355.0
25,ヘリオス テクノ ホールディング,6927,電気機器,1101200,461.0


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,794900,1219.0
1,ウェッジホールディングス,2388,その他金融業,2597000,159.0
2,ベースフード,2936,食料品,1608900,512.0
3,海帆,3133,小売業,4716800,540.0
4,アプリックス,3727,情報・通信業,6257100,188.0
5,Ａｉｍｉｎｇ,3911,情報・通信業,2533800,416.0
6,アララ,4015,情報・通信業,953200,609.0
7,プラスアルファ・コンサルティング,4071,情報・通信業,997000,2745.0
8,プレイド,4165,情報・通信業,798800,839.0
9,エクサウィザーズ,4259,情報・通信業,510000,464.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,794900,1219.0
3,海帆,3133,小売業,4716800,540.0
8,プレイド,4165,情報・通信業,798800,839.0
9,エクサウィザーズ,4259,情報・通信業,510000,464.0
10,ＨＥＮＮＧＥ,4475,情報・通信業,551000,886.0
22,ティムス,4891,医薬品,1037600,271.0
23,リプロセル,4978,化学,796400,278.0
24,ＡＮＹＣＯＬＯＲ,5032,情報・通信業,2218400,3540.0
25,カバー,5253,情報・通信業,6052500,2341.0
26,モンスターラボホールディングス,5255,情報・通信業,1184900,754.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')